<a href="https://colab.research.google.com/github/nikhomgithub/tfSciPy2018/blob/master/raw_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
  

--2019-09-14 08:20:11--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.5MB/s    in 6.6s    

2019-09-14 08:20:18 (12.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



https://github.com/the-computer-scientist/IMDBSentimentInTensorflow/blob/master/IMDBSentiment.ipynb

In [39]:
!ls -alh


total 994M
drwxr-xr-x 1 root root 4.0K Sep 14 09:19 .
drwxr-xr-x 1 root root 4.0K Sep 14 07:27 ..
-rw-r--r-- 1 root root  81M Jun 26  2011 aclImdb_v1.tar.gz
drwxr-xr-x 1 root root 4.0K Sep  9 16:05 .config
-rw-r--r-- 1 root root  92M Sep 14 09:19 glove.6B.50d.txt
-rw-r--r-- 1 root root 823M Sep 14 09:19 glove.6B.zip
drwxr-xr-x 1 root root 4.0K Aug 27 16:17 sample_data


In [5]:
!mkdir /tmp

mkdir: cannot create directory ‘/tmp’: File exists


In [0]:
!tar -xvzf aclImdb_v1.tar.gz -C /tmp

In [34]:
!ls /tmp/aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [35]:
!ls /tmp/aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [18]:
!cat /tmp/aclImdb/train/pos/2438_9.txt 

This was a gem. Amazing acting from the leads Liam Cunningham, Orla Brady and all the supporting cast. The movie raises a subject not only pertinent to Ireland and Irish history but to many communities around the world and many marriage units within those communities. With intensity and sincerity the movie shows how the religious convictions and traditions drove a wedge on a loving and passionate family. The title "Love divided" couldn't capture it any better. Even though it was a true story and happening in Ireland of the 50th seeing how the life of the whole village erodes and "pogroms" are starting reminded me of Russian history. The intolerance and prejudice are still too powerful in the world and unfortunately it's deeply hidden inside the human nature. Just like in the movie the Liam Cunningham's character says "the hatred had always been there under the surface". It was interesting to watch the moral choices people were making in this story. Also the character of a catholic prie

In [0]:
import numpy as np
import re
import os

imdb_folder='/tmp/aclImdb/'

def get_reviews(data_folder="/train"):
    reviews = []
    labels = []
    for index,sentiment in enumerate(["/neg/", "/pos/"]):
        path = imdb_folder + data_folder + sentiment
        for filename in sorted(os.listdir(path)):
            with open(path + filename, 'r') as f:
                review = f.read()
                review = review.lower()
                review = review.replace("<br />", " ")
                review = re.sub(r"[^a-z ]", " ", review)
                review = re.sub(r" +", " ", review)
                review = review.split(" ")
                reviews.append(review)
                
                label = [0, 0]
                label[index] = 1
                labels.append(label)
                
    return reviews, np.array(labels)

In [37]:
train_reviews, train_labels = get_reviews()
print(len(train_reviews))
print(train_reviews[0])
print(train_labels[0])

25000
['story', 'of', 'a', 'man', 'who', 'has', 'unnatural', 'feelings', 'for', 'a', 'pig', 'starts', 'out', 'with', 'a', 'opening', 'scene', 'that', 'is', 'a', 'terrific', 'example', 'of', 'absurd', 'comedy', 'a', 'formal', 'orchestra', 'audience', 'is', 'turned', 'into', 'an', 'insane', 'violent', 'mob', 'by', 'the', 'crazy', 'chantings', 'of', 'it', 's', 'singers', 'unfortunately', 'it', 'stays', 'absurd', 'the', 'whole', 'time', 'with', 'no', 'general', 'narrative', 'eventually', 'making', 'it', 'just', 'too', 'off', 'putting', 'even', 'those', 'from', 'the', 'era', 'should', 'be', 'turned', 'off', 'the', 'cryptic', 'dialogue', 'would', 'make', 'shakespeare', 'seem', 'easy', 'to', 'a', 'third', 'grader', 'on', 'a', 'technical', 'level', 'it', 's', 'better', 'than', 'you', 'might', 'think', 'with', 'some', 'good', 'cinematography', 'by', 'future', 'great', 'vilmos', 'zsigmond', 'future', 'stars', 'sally', 'kirkland', 'and', 'frederic', 'forrest', 'can', 'be', 'seen', 'briefly', '']


In [0]:
import urllib.request as req
import tarfile
import os
import zipfile

glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"

save_filename = "glove.6B.zip"
if not os.path.exists(save_filename):
    req.urlretrieve(glove_url, save_filename)
    
EMBEDDING_SIZE = 50

glove_filename = "glove.6B.{}d.txt".format(EMBEDDING_SIZE)
if not os.path.exists(glove_filename) and EMBEDDING_SIZE in [50,100,200,300]:
    with zipfile.ZipFile(save_filename, 'r') as z:
        z.extractall()



In [0]:
from collections import defaultdict

def load_embeddings():
    with open(glove_filename, 'r') as glove_vectors:
        word_to_int = defaultdict(int)
        int_to_vec = defaultdict(lambda: np.zeros([EMBEDDING_SIZE]))
        
        index = 1
        for line in glove_vectors:
            fields = line.split()
            word = str(fields[0])
            vec = np.asarray(fields[1:], np.float32)
            word_to_int[word] = index
            int_to_vec[index] = vec
            index += 1
    return word_to_int, int_to_vec

word_to_int, int_to_vec = load_embeddings()



In [47]:
def review_words_to_ints(train_reviews):
    train_data = []
    for review in train_reviews:
        int_review = [word_to_int[word] for word in review]
        train_data.append(int_review)
    return train_data

train_reviews = review_words_to_ints(train_reviews)
print(train_reviews[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
